# Data Cleaning and Merging

In [2]:
import pandas as pd

# Citibike Data

In [3]:
citibike_df = pd.read_csv(r"https://raw.githubusercontent.com/s-surya-s/IA-Final/main/1_Data/citibike_data.csv", sep = ",", engine = 'python').astype(str)

In [4]:
# Check Duplicates
'Duplicates Found' if citibike_df.duplicated().any() else 'No Duplicates'

'No Duplicates'

In [5]:
#Check Nulls
citibike_df.replace('nan', None, inplace=True)

'Nulls Found' if citibike_df.isnull().sum().sum() else 'No Nulls'

'Nulls Found'

In [6]:
citibike_df.isnull().sum()

ride_id                0
rideable_type          0
started_at             0
ended_at               0
start_station_name     0
start_station_id       0
end_station_name      77
end_station_id        83
start_lat              0
start_lng              0
end_lat               12
end_lng               12
member_casual          0
dtype: int64

Since the number of nulls is very low. We can drop these rows with null values

In [7]:
citibike_df = citibike_df.dropna()

'Nulls Found' if citibike_df.isnull().sum().sum() else 'No Nulls'

'No Nulls'

# Weather Data

In [8]:
weather_df = pd.read_csv(r'https://raw.githubusercontent.com/s-surya-s/IA-Final/main/1_Data/weather_data.csv')
weather_df = weather_df [['datetime', 'temp', 'icon']]

In [9]:
# Check Duplicates
'Duplicates Found' if weather_df.duplicated().any() else 'No Duplicates'

'No Duplicates'

In [10]:
#Check Nulls
weather_df.replace('nan', None, inplace=True)
'Nulls Found' if weather_df.isnull().sum().sum() != 0 else 'No Nulls'

'No Nulls'

# Merged Data

In [11]:
# Change Datatype
citibike_df['started_at'] = pd.to_datetime(citibike_df['started_at'], format='%m/%d/%Y %I:%M:%S %p')
citibike_df['ended_at']   = pd.to_datetime(citibike_df['ended_at'], format='%m/%d/%Y %I:%M:%S %p')
citibike_df['start_station_id'] = citibike_df['start_station_id'].astype(float).astype(int).astype(str)
citibike_df['end_station_id'] = citibike_df['end_station_id'].astype(float).astype(int).astype(str)

# Extract the date and hour from both DataFrames
citibike_df['ride_date'] = citibike_df['started_at'].dt.date
citibike_df['ride_hour'] = citibike_df['started_at'].dt.hour
weather_df['ride_date'] = pd.to_datetime(weather_df['datetime']).dt.date
weather_df['ride_hour'] = pd.to_datetime(weather_df['datetime']).dt.hour

merged_df = citibike_df.merge(weather_df, on=['ride_date', 'ride_hour'], how='inner')

merged_df.drop(columns='datetime', inplace=True)

merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_date,ride_hour,temp,icon
0,A47BFF9C94BC0A0C,classic_bike,2023-12-24 01:23:39,2023-12-24 01:31:33,W 82 St & Central Park West,7304,Broadway & W 56 St,6847,40.782935023,-73.97187984,40.7652654,-73.98192338,member,2023-12-24,1,6.1,partly-cloudy-night
1,11248CA60364D171,classic_bike,2023-12-24 01:18:23,2023-12-24 01:31:57,Central Park W & W 91 St,7453,East End Ave & E 86 St,7113,40.788738966,-73.966504455,40.77518615,-73.94446054,casual,2023-12-24,1,6.1,partly-cloudy-night
2,DF8B521B0B7C0B3B,classic_bike,2023-12-24 01:16:43,2023-12-24 01:29:45,Central Park W & W 91 St,7453,E 81 St & 3 Ave,7154,40.788673759,-73.966587186,40.77573034,-73.9567526,member,2023-12-24,1,6.1,partly-cloudy-night
3,C7F54940929FB07A,classic_bike,2023-12-24 01:18:15,2023-12-24 01:33:27,Central Park W & W 91 St,7453,E 84 St & 1 Ave,7180,40.788849115,-73.966679454,40.77565541,-73.95068615,casual,2023-12-24,1,6.1,partly-cloudy-night
4,8A1A2655D1078F23,classic_bike,2023-12-24 01:17:19,2023-12-24 01:29:14,Central Park W & W 91 St,7453,E 89 St & 3 Ave,7265,40.788808227,-73.966622949,40.7806284,-73.9521667,casual,2023-12-24,1,6.1,partly-cloudy-night


## Calculated Fields

In [28]:
merged_df['ride_duration'] = ((merged_df['ended_at']-merged_df['started_at']).dt.total_seconds()/60).round(2)

def calculate_fare(row):
  # Base fare for non-members
  base_fare = 4.79 if row['member_casual'] != "member" else 0

  # Ebike fare for members and casual riders
  ebike_fare = 0
  if row['rideable_type'] == "electric_bike":
    ebike_fare = row['ride_duration'] * (0.2 if row['member_casual'] == "member" else 0.3)

  # Combine fares
  return base_fare + ebike_fare

# Apply function to create 'Fare' column
merged_df['fare'] = merged_df.apply(calculate_fare, axis=1)

merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_date,ride_hour,temp,icon,ride_duration,fare
0,A47BFF9C94BC0A0C,classic_bike,2023-12-24 01:23:39,2023-12-24 01:31:33,W 82 St & Central Park West,7304,Broadway & W 56 St,6847,40.782935023,-73.97187984,40.7652654,-73.98192338,member,2023-12-24,1,6.1,partly-cloudy-night,7.90,0.00
1,11248CA60364D171,classic_bike,2023-12-24 01:18:23,2023-12-24 01:31:57,Central Park W & W 91 St,7453,East End Ave & E 86 St,7113,40.788738966,-73.966504455,40.77518615,-73.94446054,casual,2023-12-24,1,6.1,partly-cloudy-night,13.57,4.79
2,DF8B521B0B7C0B3B,classic_bike,2023-12-24 01:16:43,2023-12-24 01:29:45,Central Park W & W 91 St,7453,E 81 St & 3 Ave,7154,40.788673759,-73.966587186,40.77573034,-73.9567526,member,2023-12-24,1,6.1,partly-cloudy-night,13.03,0.00
3,C7F54940929FB07A,classic_bike,2023-12-24 01:18:15,2023-12-24 01:33:27,Central Park W & W 91 St,7453,E 84 St & 1 Ave,7180,40.788849115,-73.966679454,40.77565541,-73.95068615,casual,2023-12-24,1,6.1,partly-cloudy-night,15.20,4.79
4,8A1A2655D1078F23,classic_bike,2023-12-24 01:17:19,2023-12-24 01:29:14,Central Park W & W 91 St,7453,E 89 St & 3 Ave,7265,40.788808227,-73.966622949,40.7806284,-73.9521667,casual,2023-12-24,1,6.1,partly-cloudy-night,11.92,4.79


## Export Data

In [29]:
merged_df.to_csv(r'D:\Masters\CourseWork\Information Architecture\IA-Final\4.2_AWS_Staging_Zone\citibike_weather_merged.csv', index = False)